In [48]:
import json

combined_data_de = []

# Load data from each JSON file, extract 'en' and 'stars', and append them to the combined_data list
files = [
    # "/home/nalin/ML_metaQR/trial/translated_test_jsons/de_test_en.json",
    # "/home/nalin/ML_metaQR/trial/translated_test_jsons/es_test_en.json",
    # "/home/nalin/ML_metaQR/trial/translated_test_jsons/fr_test_en.json"
    # "/home/nalin/ML_metaQR/trial/translated_test_jsons/ja_test_en.json",
    "/home/nalin/ML_metaQR/trial/translated_test_jsons/zh_test_en.json"
]

for file_path in files:
    with open(file_path) as f:
        data = json.load(f)
        for item in data:
            combined_data_de.append({'en': item['en'], 'stars': item['stars']})



In [49]:
def remap_labels(example):
    class_mapping = {1: 0, 2: 1, 4: 2, 5: 3}
    example['stars'] = class_mapping.get(example['stars'])  
    return example


combined_data_test = [remap_labels(example) for example in combined_data_de]




In [50]:
from torch.utils.data import Dataset
import torch
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Tokenize text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Extract input IDs and attention masks
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [51]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

# Load the pre-trained model for sequence classification
model = BertForSequenceClassification.from_pretrained("/home/nalin/ML_metaQR/trial/translated_bert/translated_bert_model_stars")
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()  


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


test_dataset = CustomDataset([item['en'] for item in combined_data_de], [item['stars'] for item in combined_data_de], tokenizer, 512)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

test_predicted = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        test_predicted.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

test_f1 = f1_score(test_labels, test_predicted, average='weighted')
accuracy = accuracy_score(test_labels, test_predicted)

# Print test F1 score and accuracy
print(f"Test F1 Score: {test_f1}")
print(f"Test Accuracy: {accuracy}")


Testing: 100%|██████████| 125/125 [01:06<00:00,  1.89batch/s]

Test F1 Score: 0.540879460997924
Test Accuracy: 0.542875
